In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
import random
from keras.preprocessing.image import load_img

print (tf.__version__)

In [ ]:
train_dir = "../input/100-bird-species/train"
test_dir = "../input/100-bird-species/test"
valid_dir = "../input/100-bird-species/valid"

In [ ]:
labels = os.listdir(train_dir)
print(labels)

In [ ]:
height = 224
width = 224
batch_size = 128

train_datagen = keras.preprocessing.image.ImageDataGenerator( 
                                            rescale=1./255, 
                                            rotation_range=40, 
                                            width_shift_range=0.2,
                                            height_shift_range=0.2, 
                                            zoom_range=0.2,
                                            horizontal_flip=True, 
                                            fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
                                            train_dir,
                                            target_size = (height, width),
                                            batch_size = batch_size,
                                            shuffle=True,
                                            class_mode='categorical'
)

test_datagen = keras.preprocessing.image.ImageDataGenerator(
                                            rescale = 1./255
)

test_generator = test_datagen.flow_from_directory(
                                            test_dir,
                                            target_size = (height, width),
                                            batch_size = batch_size,
                                            shuffle = False,
                                            class_mode = "categorical"
)

valid_datagen = keras.preprocessing.image.ImageDataGenerator(
                                            rescale = 1./255
)

valid_generator = valid_datagen.flow_from_directory(
                                            valid_dir,
                                            target_size = (height, width),
                                            batch_size = batch_size,
                                            shuffle = False,
                                            class_mode = "categorical"
)

train_num = train_generator.samples
test_num = test_generator.samples
valid_num = valid_generator.samples

print(train_num, test_num, valid_num)

In [ ]:
def label_images2(DIR, dataset):
    label = []
    image = []
    j=0
    for i in range (0,30):
        j = random.randint(0, len(dataset.filenames))
        label.append(dataset.filenames[j].split('/')[0])
        image.append(DIR + '/' + dataset.filenames[j])
    return [label,image]

#plot the random images.
y,x = label_images2(train_dir,train_generator )

for i in range(0,6):
    X = load_img(x[i])
    plt.subplot(2,3,+1 + i)
    plt.axis(False)
    plt.title(y[i], fontsize=8)
    plt.imshow(X)
plt.show()

In [ ]:
from pandas.core.frame import DataFrame
import pandas as pd

bird_name = []
train_n = []
test_n = []
valid_n = []

for bird in os.listdir(train_dir):
    count_train = 0
    for dir in os.listdir(train_dir + '/' + bird):
        count_train = count_train + 1
    count_test = 0
    for dir in os.listdir(test_dir + '/' + bird):
        count_test = count_test + 1
    count_valid = 0
    for dir in os.listdir(valid_dir + '/' + bird):
        count_valid = count_valid + 1
    bird_name.append(bird)
    train_n.append(count_train)
    test_n.append(count_test)
    valid_n.append(count_valid)
list_bird = {"Bird Name":bird_name,"Train Number":train_n,"Test Number":test_n,"Valid Number":valid_n}
data = DataFrame(list_bird)
print(data)

In [ ]:
covn_base = tf.keras.applications.xception.Xception(weights='imagenet',include_top=False)
covn_base.trainable = True

for layers in covn_base.layers[:-32]:
    layers.trainable = False

model = tf.keras.Sequential()
model.add(covn_base)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(230))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])
model.summary()

In [ ]:
epochs = 10

Early_sp = EarlyStopping(monitor = 'val_accuracy', patience = 5,restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)

history = model.fit(
                    train_generator,
                    steps_per_epoch = train_num // batch_size,
                    epochs = epochs,
                    validation_data = valid_generator,
                    validation_steps = valid_num // batch_size,
                    callbacks = [Early_sp,reduce_lr]
)

In [ ]:
history_dict = history.history
train_loss = history_dict["loss"]
train_accuracy = history_dict["accuracy"]
val_loss = history_dict["val_loss"]
val_accuracy = history_dict["val_accuracy"]

# figure 1
plt.figure()
plt.plot(range(epochs), train_loss, label='train_loss')
plt.plot(range(epochs), val_loss, label='val_loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')

# figure 2
plt.figure()
plt.plot(range(epochs), train_accuracy, label='train_accuracy')
plt.plot(range(epochs), val_accuracy, label='val_accuracy')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
scores = model.evaluate(test_generator, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])